## Station Location Features

In [1]:
import pandas as pd
import Useful_functions_for_stations
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap


In [2]:
bluebikes = pd.read_csv('C:/Users/lauri/PycharmProjects/DSML_Data/bluebikes_idle_time_v4.csv')
bluebikes

,start_time,end_time,start_station_id,end_station_id,bike_id,weekday,idle_time
0,2017-01-01 00:06:58,2017-01-01 00:12:49,67,139,644,6,0
1,2017-01-01 00:13:16,2017-01-01 00:28:07,36,10,230,6,0
2,2017-01-01 00:16:17,2017-01-01 00:44:10,36,9,980,6,0
3,2017-01-01 00:21:22,2017-01-01 00:33:50,46,19,1834,6,0
4,2017-01-01 00:30:06,2017-01-01 00:40:28,10,8,230,6,0 days 00:01:59
...,...,...,...,...,...,...,...
3081456,2018-12-31 23:05:54,2018-12-31 23:08:31,43,40,3050,0,0 days 05:03:27
3081457,2018-12-31 23:31:50,2018-12-31 23:39:14,107,90,2673,0,0 days 06:00:40
3081458,2018-12-31 23:48:46,2018-12-31 23:56:55,19,179,4440,0,0 days 06:07:52
3081459,2018-12-31 23:54:16,2018-12-31 23:57:17,352,36,3605,0,0 days 06:59:37


In [3]:
bluebikes_stations = pd.read_csv('C:/Users/lauri/PycharmProjects/DSML_Data/bluebikes_stations.csv')
bluebikes_stations_dist = bluebikes_stations.copy()
bluebikes_stations.head()

,station_id,lat,lon
0,67,42.358100,-71.093198
1,139,42.361780,-71.108100
2,36,42.349792,-71.077338
3,10,42.350406,-71.108279
4,9,42.351187,-71.115852


Density-based feature: Calculate density of stations within a certain radius around each station. High density might imply high competition which could lead to higher idle times.

In [6]:
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic

C:\Users\lauri\AppData\Local\Temp\ipykernel_19016\3713068787.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [7]:
geodata = gpd.GeoDataFrame(bluebikes_stations, geometry = gpd.points_from_xy(bluebikes_stations.lon, bluebikes_stations.lat))

geodata.set_crs(epsg=4326, inplace=True)

geodata = geodata.to_crs(epsg=3857)

buffer_radius = 1000 # = 1000 meter

geodata['buffer'] = geodata['geometry'].buffer(buffer_radius)
# Calculate number of stations within the radius around a station
geodata['station_density'] = geodata.apply(lambda row: geodata.within(row['buffer']).sum() - 1, axis=1)
# Convert station_density to per square km
geodata['station_density'] = geodata['station_density'] / (3.1416* (buffer_radius/1000)**2)



In [8]:
geodata

,station_id,lat,lon,geometry,buffer,station_density
0,67,42.358100,-71.093198,POINT (-7914058.600 5214772.943),"POLYGON ((-7913058.600 5214772.943, -7913063.4...",0.954927
1,139,42.361780,-71.108100,POINT (-7915717.430 5215327.402),"POLYGON ((-7914717.430 5215327.402, -7914722.2...",2.546473
2,36,42.349792,-71.077338,POINT (-7912293.113 5213521.392),"POLYGON ((-7911293.113 5213521.392, -7911297.9...",2.864782
3,10,42.350406,-71.108279,POINT (-7915737.409 5213613.945),"POLYGON ((-7914737.409 5213613.945, -7914742.2...",1.591546
4,9,42.351187,-71.115852,POINT (-7916580.435 5213731.525),"POLYGON ((-7915580.435 5213731.525, -7915585.2...",1.273237
...,...,...,...,...,...,...
308,229,42.345033,-71.096649,POINT (-7914442.783 5212804.682),"POLYGON ((-7913442.783 5212804.682, -7913447.5...",3.501401
309,374,42.356683,-71.061666,POINT (-7910548.525 5214559.533),"POLYGON ((-7909548.525 5214559.533, -7909553.3...",4.456328
310,376,42.360274,-71.128525,POINT (-7917991.131 5215100.408),"POLYGON ((-7916991.131 5215100.408, -7916995.9...",0.636618
311,377,42.379273,-71.103419,POINT (-7915196.400 5217963.142),"POLYGON ((-7914196.400 5217963.142, -7914201.2...",2.228164


Distance-based feature: There are important locations in Boston like the City_center,the universities or popular train stations. Bike stations closer to those locations might be used more frequently which could lead to lower idle times

In [9]:
# Important locations in Boston
locations = {
    'city_center': (42.360082, -71.058880), # Boston City Center
    'boston_university': (42.350503, -71.105399), # Boston University
    'fenway_park': (42.346676, -71.097218), # Fenway Park
    'harvard_university': (42.373611, -71.109733), # Harvard University
    'mit': (42.360091, -71.094160), # Massachusetts Institute of Technology
    'boston_common': (42.355443, -71.066163), # Boston Common: public park in downtown
    'boston_public_garden': (42.353865, -71.071342), # Boston Public Garden
    'newbury_street': (42.349846, -71.083415), # Newbury Street (shopping district)
    'south_station': (42.352175, -71.055242), # South Station (major train/subway station)
    'north_station': (42.365577, -71.061245), # North Station (major train/subway station)
    'back_bay_station': (42.347351, -71.075187), # Back Bay Station (major train/subway station)
}
# Calculate distance to each location
for loc_name, loc_coord in locations.items():
    bluebikes_stations_dist[loc_name + '_distance'] = bluebikes_stations_dist.apply(lambda row: geodesic((row['lat'], row['lon']), loc_coord).km, axis=1)

bluebikes_stations_dist

,station_id,lat,lon,city_center_distance,boston_university_distance,fenway_park_distance,harvard_university_distance,mit_distance,boston_common_distance,boston_public_garden_distance,newbury_street_distance,south_station_distance,north_station_distance,back_bay_station_distance
0,67,42.358100,-71.093198,2.835790,1.312488,1.311493,2.196322,0.234932,2.246784,1.861094,1.220776,3.195646,2.760200,1.904668
1,139,42.361780,-71.108100,4.059136,1.272305,1.902310,1.321014,1.163583,3.525886,3.153329,2.427624,4.483532,3.882721,3.149924
2,36,42.349792,-71.077338,1.902449,2.313407,1.674192,3.757982,1.797120,1.114398,0.669948,0.500724,1.839755,2.198291,0.323908
3,10,42.350406,-71.108279,4.209433,0.237541,1.001153,2.580400,1.584456,3.514814,3.067487,2.049614,4.374321,4.225473,2.747706
4,9,42.351187,-71.115852,4.796679,0.864608,1.615064,2.541403,2.042611,4.121133,3.679314,2.676779,4.995049,4.774306,3.377621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,229,42.345033,-71.096649,3.532434,0.942848,0.188411,3.352443,1.685154,2.765277,2.304460,1.214474,3.502844,3.703488,1.787130
309,374,42.356683,-71.061666,0.441839,3.667965,3.133033,4.383297,2.703585,0.395249,0.856422,1.946195,0.728667,0.988519,1.521709
310,376,42.360274,-71.128525,5.737488,2.192714,2.989070,2.142643,2.831087,5.165596,4.764520,3.892831,6.104186,5.573593,4.622995
311,377,42.379273,-71.103419,4.243053,3.199969,3.656769,0.816101,2.263149,4.052775,3.866193,3.660657,4.980976,3.792258,4.240577


Stations with lowest distance to north station: 22, 192 59 35 64 48 157 81 365 302 43 151 374 54 23 218 20 7 24 49

,station_id,lat,lon,mit_distance
0,67,42.358100,-71.093198,0.234932
1,139,42.361780,-71.108100,1.163583
2,36,42.349792,-71.077338,1.797120
3,10,42.350406,-71.108279,1.584456
4,9,42.351187,-71.115852,2.042611
...,...,...,...,...
308,229,42.345033,-71.096649,1.685154
309,374,42.356683,-71.061666,2.703585
310,376,42.360274,-71.128525,2.831087
311,377,42.379273,-71.103419,2.263149


In [12]:
# Convert idle_time to timedelta type
bluebikes['idle_time'] = pd.to_timedelta(bluebikes['idle_time'])

# Convert idle_time to minutes
bluebikes['idle_time_hours'] = bluebikes['idle_time'].dt.total_seconds() / 3600


In [18]:
bluebikes.quantile(0.95)
# Remove highest idle times: top 5%
bluebikes = bluebikes[bluebikes['idle_time'] <= pd.Timedelta(hours=8)]

C:\Users\lauri\AppData\Local\Temp\ipykernel_19016\2153567739.py:1: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  bluebikes.quantile(0.95)


Merge the two tables and compare the idle times with the calculated distances.

In [19]:
merge = pd.merge(bluebikes_stations_dist, bluebikes, left_on='station_id', right_on='start_station_id', how='inner')
merge

,station_id,lat,lon,city_center_distance,boston_university_distance,fenway_park_distance,harvard_university_distance,mit_distance,boston_common_distance,boston_public_garden_distance,...,north_station_distance,back_bay_station_distance,start_time,end_time,start_station_id,end_station_id,bike_id,weekday,idle_time,idle_time_hours
0,67,42.358100,-71.093198,2.835790,1.312488,1.311493,2.196322,0.234932,2.246784,1.861094,...,2.760200,1.904668,2017-01-01 00:06:58,2017-01-01 00:12:49,67,139,644,6,0 days 00:00:00,0.000000
1,67,42.358100,-71.093198,2.835790,1.312488,1.311493,2.196322,0.234932,2.246784,1.861094,...,2.760200,1.904668,2017-01-01 00:42:04,2017-01-01 00:46:25,67,179,1102,6,0 days 00:00:00,0.000000
2,67,42.358100,-71.093198,2.835790,1.312488,1.311493,2.196322,0.234932,2.246784,1.861094,...,2.760200,1.904668,2017-01-01 03:00:00,2017-01-01 03:03:45,67,178,972,6,0 days 01:34:03,1.567500
3,67,42.358100,-71.093198,2.835790,1.312488,1.311493,2.196322,0.234932,2.246784,1.861094,...,2.760200,1.904668,2017-01-01 04:03:37,2017-01-01 04:24:27,67,67,532,6,0 days 02:37:40,2.627778
4,67,42.358100,-71.093198,2.835790,1.312488,1.311493,2.196322,0.234932,2.246784,1.861094,...,2.760200,1.904668,2017-01-01 11:10:04,2017-01-01 11:31:10,67,74,532,6,0 days 00:12:15,0.204167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915702,378,42.380323,-71.108786,4.685436,3.324186,3.857133,0.749679,2.549958,4.468208,4.260417,...,4.244542,4.590774,2018-12-30 19:11:40,2018-12-30 19:16:33,378,115,2644,6,0 days 03:46:21,3.772500
2915703,378,42.380323,-71.108786,4.685436,3.324186,3.857133,0.749679,2.549958,4.468208,4.260417,...,4.244542,4.590774,2018-12-31 16:33:23,2018-12-31 16:43:30,378,145,3988,0,0 days 01:25:14,1.420556
2915704,378,42.380323,-71.108786,4.685436,3.324186,3.857133,0.749679,2.549958,4.468208,4.260417,...,4.244542,4.590774,2018-12-31 18:40:51,2018-12-31 18:47:26,378,78,2563,0,0 days 00:26:08,0.435556
2915705,378,42.380323,-71.108786,4.685436,3.324186,3.857133,0.749679,2.549958,4.468208,4.260417,...,4.244542,4.590774,2018-12-31 18:58:43,2018-12-31 19:27:07,378,338,3838,0,0 days 00:44:00,0.733333


In [20]:
merge.corr()

C:\Users\lauri\AppData\Local\Temp\ipykernel_19016\3416459868.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  merge.corr()


,station_id,lat,lon,city_center_distance,boston_university_distance,fenway_park_distance,harvard_university_distance,mit_distance,boston_common_distance,boston_public_garden_distance,newbury_street_distance,south_station_distance,north_station_distance,back_bay_station_distance,start_station_id,end_station_id,bike_id,weekday,idle_time_hours
station_id,1.000000,0.227137,-0.079473,0.143457,0.102195,0.205662,-0.120775,0.015746,0.194244,0.216748,0.253502,0.190203,0.082710,0.282858,1.000000,0.134083,0.074732,0.001802,0.090760
lat,0.227137,1.000000,-0.303793,0.180773,0.188476,0.444405,-0.618776,-0.108097,0.305543,0.356628,0.471670,0.375405,0.018009,0.561088,0.227137,0.139958,-0.012724,-0.000290,-0.012335
lon,-0.079473,-0.303793,1.000000,-0.869454,0.539931,0.231742,0.737011,0.284532,-0.795562,-0.692229,-0.293176,-0.912287,-0.802582,-0.579778,-0.079473,-0.043319,-0.003481,-0.034883,-0.033328
city_center_distance,0.143457,0.180773,-0.869454,1.000000,-0.226415,0.054381,-0.443221,0.112374,0.968166,0.905673,0.570378,0.964646,0.977816,0.761940,0.143457,0.055412,0.012204,0.041698,0.140426
boston_university_distance,0.102195,0.188476,0.539931,-0.226415,1.000000,0.921524,0.584006,0.835173,-0.042732,0.125558,0.583453,-0.248410,-0.195872,0.281377,0.102195,0.032217,0.006745,-0.010131,0.143725
fenway_park_distance,0.205662,0.444405,0.231742,0.054381,0.921524,1.000000,0.284812,0.777676,0.258610,0.422541,0.820073,0.074818,0.039790,0.597883,0.205662,0.082170,0.005133,-0.001237,0.162564
harvard_university_distance,-0.120775,-0.618776,0.737011,-0.443221,0.584006,0.284812,1.000000,0.669082,-0.399343,-0.310754,-0.036760,-0.593886,-0.299005,-0.336292,-0.120775,-0.089461,0.015862,-0.014498,0.093528
mit_distance,0.015746,-0.108097,0.284532,0.112374,0.835173,0.777676,0.669082,1.000000,0.248087,0.382290,0.669121,0.002089,0.204237,0.399785,0.015746,-0.027394,0.018131,0.007552,0.210342
boston_common_distance,0.194244,0.305543,-0.795562,0.968166,-0.042732,0.258610,-0.399343,0.248087,1.000000,0.980995,0.738600,0.956446,0.932622,0.890299,0.194244,0.078903,0.013510,0.037198,0.163861
boston_public_garden_distance,0.216748,0.356628,-0.692229,0.905673,0.125558,0.422541,-0.310754,0.382290,0.980995,1.000000,0.849969,0.898212,0.872134,0.949887,0.216748,0.086807,0.014501,0.032597,0.182193


Merge station_density data with the idle time to see if they correlate

In [21]:
merge_geodata = pd.merge(geodata, bluebikes, left_on='station_id', right_on='start_station_id', how='inner')
merge_geodata

,station_id,lat,lon,geometry,buffer,station_density,start_time,end_time,start_station_id,end_station_id,bike_id,weekday,idle_time,idle_time_hours
0,67,42.358100,-71.093198,POINT (-7914058.600 5214772.943),"POLYGON ((-7913058.600 5214772.943, -7913063.4...",0.954927,2017-01-01 00:06:58,2017-01-01 00:12:49,67,139,644,6,0 days 00:00:00,0.000000
1,67,42.358100,-71.093198,POINT (-7914058.600 5214772.943),"POLYGON ((-7913058.600 5214772.943, -7913063.4...",0.954927,2017-01-01 00:42:04,2017-01-01 00:46:25,67,179,1102,6,0 days 00:00:00,0.000000
2,67,42.358100,-71.093198,POINT (-7914058.600 5214772.943),"POLYGON ((-7913058.600 5214772.943, -7913063.4...",0.954927,2017-01-01 03:00:00,2017-01-01 03:03:45,67,178,972,6,0 days 01:34:03,1.567500
3,67,42.358100,-71.093198,POINT (-7914058.600 5214772.943),"POLYGON ((-7913058.600 5214772.943, -7913063.4...",0.954927,2017-01-01 04:03:37,2017-01-01 04:24:27,67,67,532,6,0 days 02:37:40,2.627778
4,67,42.358100,-71.093198,POINT (-7914058.600 5214772.943),"POLYGON ((-7913058.600 5214772.943, -7913063.4...",0.954927,2017-01-01 11:10:04,2017-01-01 11:31:10,67,74,532,6,0 days 00:12:15,0.204167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915702,378,42.380323,-71.108786,POINT (-7915793.863 5218121.391),"POLYGON ((-7914793.863 5218121.391, -7914798.6...",2.546473,2018-12-30 19:11:40,2018-12-30 19:16:33,378,115,2644,6,0 days 03:46:21,3.772500
2915703,378,42.380323,-71.108786,POINT (-7915793.863 5218121.391),"POLYGON ((-7914793.863 5218121.391, -7914798.6...",2.546473,2018-12-31 16:33:23,2018-12-31 16:43:30,378,145,3988,0,0 days 01:25:14,1.420556
2915704,378,42.380323,-71.108786,POINT (-7915793.863 5218121.391),"POLYGON ((-7914793.863 5218121.391, -7914798.6...",2.546473,2018-12-31 18:40:51,2018-12-31 18:47:26,378,78,2563,0,0 days 00:26:08,0.435556
2915705,378,42.380323,-71.108786,POINT (-7915793.863 5218121.391),"POLYGON ((-7914793.863 5218121.391, -7914798.6...",2.546473,2018-12-31 18:58:43,2018-12-31 19:27:07,378,338,3838,0,0 days 00:44:00,0.733333


In [22]:
merge_geodata.corr()

C:\Users\lauri\AppData\Local\Temp\ipykernel_19016\2671976427.py:1: FutureWarning: The default value of numeric_only in GeoDataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  merge_geodata.corr()


,station_id,lat,lon,station_density,start_station_id,end_station_id,bike_id,weekday,idle_time_hours
station_id,1.000000,0.227137,-0.079473,-0.216896,1.000000,0.134083,0.074732,0.001802,0.090760
lat,0.227137,1.000000,-0.303793,-0.132932,0.227137,0.139958,-0.012724,-0.000290,-0.012335
lon,-0.079473,-0.303793,1.000000,0.366376,-0.079473,-0.043319,-0.003481,-0.034883,-0.033328
station_density,-0.216896,-0.132932,0.366376,1.000000,-0.216896,-0.069897,-0.006010,-0.034996,-0.141145
start_station_id,1.000000,0.227137,-0.079473,-0.216896,1.000000,0.134083,0.074732,0.001802,0.090760
end_station_id,0.134083,0.139958,-0.043319,-0.069897,0.134083,1.000000,0.079487,-0.008775,-0.000156
bike_id,0.074732,-0.012724,-0.003481,-0.006010,0.074732,0.079487,1.000000,0.013807,-0.025096
weekday,0.001802,-0.000290,-0.034883,-0.034996,0.001802,-0.008775,0.013807,1.000000,0.014358
idle_time_hours,0.090760,-0.012335,-0.033328,-0.141145,0.090760,-0.000156,-0.025096,0.014358,1.000000
